In [11]:
from ngsolve import *
from ngsolve.webgui import Draw
import ngsolve.solvers
from netgen.occ import *


In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

In [3]:
fes = H1(mesh, order=5, complex=True)
u,v = fes.TnT()

omega = 2*pi*10

a = BilinearForm(fes)
a += (grad(u)*grad(v)-omega**2*u*v)*dx
a += -1j*omega*u*v*ds
a.Assemble()

source = exp(-50**2*((x-0.5)*(x-0.5)+(y-0.5)*(y-0.5)))
f = LinearForm(source*v*dx(bonus_intorder=5)).Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (1e3*gfu, mesh, order=3, animate_complex=True, deformation=True);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

In [4]:
from netgen.occ import *
shape = MoveTo(-1,-1).Rectangle (2,2).Face()
shape.edges.Min(X).name="left"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(X).name="right"
shape.edges.Max(Y).name="top"
circ = Circle ( (0.6, 0.0), 0.07).Face()
rect = MoveTo(-0.2,-0.4).Rotate(45).Rectangle(1,0.1).Face()
# shape = shape-circ
# shape = shape-rect
mesh = Mesh (OCCGeometry( shape, dim=2).GenerateMesh(maxh=0.05)).Curve(3)
Draw (shape);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [5]:

fes = H1(mesh, order=5, complex=True)
u,v = fes.TnT()

a = BilinearForm(fes)
f = LinearForm(fes)

omega = 20*pi
outer = "left|right|bottom|top"
a += grad(u)*grad(v)*dx-omega**2*u*v*dx - 1j*omega*u*v*ds(outer)
f = LinearForm(fes)

incoming = 100*exp(-5**2*y*y)
f += incoming*v * ds("left")

a.Assemble()
f.Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu, order=3, animate_complex=True);



WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

## Eigenvalue problems

In [6]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

fes = H1(mesh, order=3, dirichlet=".*")
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
m = BilinearForm(u*v*dx).Assemble()

In [8]:
pre = a.mat.Inverse(freedofs=fes.FreeDofs())
evals, evecs = solvers.LOBPCG(a.mat, m.mat, pre=pre, num=12, maxit=20, printrates=True)

for lam in evals:
    print (lam/pi**2)

0 : [19.843798444331448, 55.68793044670485, 63.137498855438444, 118.26215364257946, 140.65960160853936, 186.6444244813282, 288.6653945238204, 318.3628476949989, 373.2853748040036, 445.4802818311113, 630.8995813816706, 739.8060575228986]
1 : [19.739337079250028, 49.399682484958525, 49.46878292598795, 79.81890062527312, 99.55078034444976, 102.34102446819149, 135.98858183857476, 139.55364065046618, 184.47709263540926, 198.67462375372406, 217.57008901504068, 305.3663626411378]
2 : [19.739209952025583, 49.348201208792545, 49.34847138718346, 78.96774503452443, 98.70839714073557, 98.77499316495759, 128.52476979730594, 128.92668835150045, 169.98007518036476, 177.51735019357588, 192.78260661103948, 198.66659295445422]
3 : [19.73920989037998, 49.34805440643043, 49.34806506104502, 78.95713499446605, 98.69677733741706, 98.69734129579412, 128.3102957237539, 128.3288837303492, 167.88421176100562, 169.68145914702725, 178.66370770240852, 197.48486324784568]
4 : [19.7392098903627, 49.34805395751565, 49

In [9]:
gfu = GridFunction(fes, multidim=0)
for evec in evecs:
    gfu.AddMultiDimComponent(evec)

Draw (gfu, min=-2, max=2);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

## compute eigenfunctions on more complex domains.

In [12]:
square = MoveTo(-1,-1).Rectangle(2,2).Face()
square.edges.name = "outer_boundary"

In [13]:
hole = Circle((0,0), 0.3).Face()
hole.edges.name = "inner_boundary"

In [16]:
shape_with_hole = square - hole
geo = OCCGeometry(shape_with_hole, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.08)).Curve(3) 
Draw(shape_with_hole);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [17]:
fes = H1(mesh, order=3, dirichlet="outer_boundary|inner_boundary")
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
m = BilinearForm(u*v*dx).Assemble()
pre = a.mat.Inverse(freedofs=fes.FreeDofs())
evals, evecs = solvers.LOBPCG(a.mat, m.mat, pre=pre, num=12, maxit=20, printrates=True)

for lam in evals:
    print (lam/pi**2)

0 : [15.102162539041469, 17.5666208946642, 19.87373400301322, 25.282790175300057, 32.65183785313701, 39.29923360202199, 47.60919574774007, 76.14090698855948, 92.19713420624609, 136.13726297220984, 142.29018567001322, 186.75819937310678]
1 : [15.080789507112101, 17.23327349890952, 17.29870641329402, 21.421165174778224, 27.924337453467537, 34.97178095231485, 35.3900853489388, 45.24992377294403, 57.377981020417025, 64.28171818020112, 70.83329877571188, 82.44020552876069]
2 : [15.080460918598872, 17.23123256749318, 17.231575553516976, 21.313316808129795, 27.6185620368936, 34.772146805570806, 34.786929705042205, 42.71788690141103, 50.85685694438353, 54.327873923396304, 56.63866642069209, 58.33653602872293]
3 : [15.08046042261286, 17.231222927080264, 17.231223999288318, 21.312130452032214, 27.61285112771349, 34.7666575458181, 34.7669356607734, 42.6189783799312, 49.65970558959213, 53.930449114264526, 54.055192937287195, 56.42745492961257]
4 : [15.080460421914037, 17.231222877027843, 17.231223

In [18]:
gfu = GridFunction(fes, multidim=0)
for evec in evecs:
    gfu.AddMultiDimComponent(evec)

Draw (gfu, min=-2, max=2);fes = H1(mesh, order=3, dirichlet="inner_boundary")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [19]:
fes = H1(mesh, order=3, dirichlet="inner_boundary")
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
m = BilinearForm(u*v*dx).Assemble()
pre = a.mat.Inverse(freedofs=fes.FreeDofs())
evals, evecs = solvers.LOBPCG(a.mat, m.mat, pre=pre, num=12, maxit=20, printrates=True)

for lam in evals:
    print (lam/pi**2)

0 : [2.0263272709915134, 3.2033136433186846, 3.215688202554016, 5.179282330716162, 11.302828692757721, 15.579145290966203, 17.493787661269565, 20.535697868188045, 30.495669293306673, 34.6857169873754, 59.16150091111529, 67.48879871731772]
1 : [2.026320765799676, 3.2010959514769555, 3.2011117284015076, 5.073140001935704, 10.394205068322734, 13.428732987442523, 13.728595674171567, 16.024035477128997, 25.46605249114355, 25.87748320238786, 27.370995945568698, 33.72644889202026]
2 : [2.0263207622658905, 3.201093074497791, 3.20109308442098, 5.072544712874494, 10.365360064503687, 13.340163472577965, 13.344295504444998, 15.863691751382293, 24.736160834716788, 25.298833411862567, 25.373304013630577, 26.48844718171632]
3 : [2.026320762265236, 3.2010930690578343, 3.201093080068866, 5.072544055307058, 10.365225324539887, 13.339476561842522, 13.339521213538323, 15.861163647705025, 24.69741677048885, 25.287558862922054, 25.291081755118483, 26.18100346608817]
4 : [2.026320762265236, 3.201093069055757

In [21]:
gfu = GridFunction(fes, multidim=0)
for evec in evecs:
    gfu.AddMultiDimComponent(evec)

Draw (gfu, min=-2, max=2);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…